In [1]:
from models.cifar_resnet import cifar_resnet

In [2]:
cresnet8 = cifar_resnet(variant=8)
cresnet18 = cifar_resnet(variant=20)
cresnet56 = cifar_resnet(variant=56)

In [3]:
no_layers = 0
for name, parameter in cresnet8.named_parameters():
    if("conv" in name): 
        no_layers += 1
print(f'No of layers = {no_layers}')

No of layers = 7


In [4]:
no_layers = 0
for name, parameter in cresnet18.named_parameters():
    if("conv" in name): 
        no_layers += 1
print(f'No of layers = {no_layers}')

No of layers = 19


In [5]:
no_layers = 0
for name, parameter in cresnet56.named_parameters():
    if("conv" in name): 
        no_layers += 1
print(f'No of layers = {no_layers}')

No of layers = 82


In [6]:
import re

def extract_int(inp_str: str) -> int:
    num = re.findall(r'\d+$',inp_str)
    if(len(num)==0):
        raise ValueError(f'{inp_str} is an incorrect argument!')
    num = int(num[0])
    return num

In [7]:
a = extract_int("resnet18")
b = extract_int("cifar100")
print(f'{a} , {b}')

18 , 100


In [8]:
from models import init_model

model1 = init_model("cifar10","resnet8")
model2 = init_model("imagenet","resnet18")


In [9]:
print(f'Shape model 1 fc : {model1.fc}')
print(f'Shape model 2 fc : {model2.fc}')

Shape model 1 fc : Linear(in_features=64, out_features=10, bias=True)
Shape model 2 fc : Linear(in_features=512, out_features=1000, bias=True)
